# 🩺 MediMate - Enhanced AI Medical Consultation System (with Audio Input)

Welcome to the latest version of MediMate, now featuring **voice input**. You can now describe your symptoms by speaking directly into your microphone, making it easier and faster to get a medical consultation.

## Key Enhancements in This Version

1.  **🎤 Audio Input with Whisper API**
    *   A new **Record Audio** button has been added to the interface.
    *   Your recorded audio is transcribed into text using **OpenAI's Whisper API**.
    *   The transcribed text is then sent to the AI, seamlessly integrating with the existing chat logic.

2.  **✨ Streamlined UI**
    *   The text and audio inputs are presented as clear alternatives.
    *   Error handling is included for cases where the microphone is not available or the API key is missing.

3.  **⚙️ Robust Backend**
    *   The `AIService` client is now reused for both chat completions and audio transcriptions.
    *   The implementation is modular, with a dedicated function for transcription that feeds into the main chat handler.

## 1. Install Required Packages

First, let's install all the necessary packages. This cell includes all dependencies for the project.

In [ ]:
# Install required packages
!pip install ollama gradio>=4.0.0 requests python-dotenv openai>=1.0.0 colorama langdetect pydantic>=2.0.0 markdown fpdf2

In [ ]:
# Import standard libraries
import os
import sys
import time
import json
import requests
import subprocess
import logging
from typing import List, Tuple, Generator, Dict, Any, Optional, Union
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger("MediMate")

# Import third-party libraries
try:
    import gradio as gr
    from dotenv import load_dotenv
    from openai import OpenAI
    from colorama import Fore, Style
    from langdetect import detect
    import markdown
    from fpdf import FPDF
    import ollama
except ImportError as e:
    logger.error(f"❌ Import error: {e}")
    logger.warning("Some packages may need to be installed. Please run the pip install command above.")

# Load environment variables
load_dotenv()

# Special handling for Ollama
try:
    import ollama
except ImportError:
    logger.warning("⚠️ Ollama import failed. Local models will not be available.")
    ollama = None

## 2. Configuration Settings

The `Config` class defines all configuration parameters. It now includes a setting for the directory where conversations will be saved.

In [ ]:
class Config:
    """Configuration settings for MediMate"""
    
    # Ollama settings
    OLLAMA_HOST = os.getenv("OLLAMA_HOST", "http://localhost:11434")
    DEFAULT_MODEL = os.getenv("OLLAMA_MODEL", "gemma:7b")
    
    # OpenRouter settings
    OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
    OPENROUTER_MODEL = os.getenv("OPENROUTER_MODEL", "meta-llama/llama-3.2-9b-instruct:free")
    OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1")
    WHISPER_MODEL = os.getenv("WHISPER_MODEL", "openai/whisper-large-v3") # Whisper model for transcription
    
    # App settings
    SERVER_PORT = int(os.getenv("SERVER_PORT", "7860"))
    AI_METHOD = os.getenv("AI_METHOD", "auto")
    DEBUG_MODE = os.getenv("DEBUG_MODE", "false").lower() == "true"
    MAX_HISTORY = int(os.getenv("MAX_HISTORY", "10"))
    CONVERSATION_DIR = os.getenv("CONVERSATION_DIR", "consultations")
    
    # Advanced features
    ENABLE_MULTILINGUAL = os.getenv("ENABLE_MULTILINGUAL", "true").lower() == "true"
    DEFAULT_LANGUAGE = os.getenv("DEFAULT_LANGUAGE", "en")
    ENABLE_PDF_EXPORT = os.getenv("ENABLE_PDF_EXPORT", "true").lower() == "true"
    
    # Medical features
    CLINICAL_DETAIL_LEVEL = os.getenv("CLINICAL_DETAIL_LEVEL", "high")
    EMERGENCY_MODE = os.getenv("EMERGENCY_MODE", "true").lower() == "true"
    
    @classmethod
    def display(cls):
        """Display current configuration settings"""
        logger.info("🔧 MediMate Configuration:")
        logger.info(f"   🖥️  Ollama: {cls.OLLAMA_HOST} ({cls.DEFAULT_MODEL})")
        logger.info(f"   🌐 OpenRouter: {cls.OPENROUTER_MODEL}")
        logger.info(f"   🎤 Whisper Model: {cls.WHISPER_MODEL}")
        logger.info(f"   ⚙️  Method: {cls.AI_METHOD}")
        logger.info(f"   🔐 API Key: {'✅ Available' if cls.OPENROUTER_API_KEY else '❌ Missing'}")
        logger.info(f"   🌍 Multilingual: {'✅ Enabled' if cls.ENABLE_MULTILINGUAL else '❌ Disabled'}")
        logger.info(f"   📄 PDF Export: {'✅ Enabled' if cls.ENABLE_PDF_EXPORT else '❌ Disabled'}")
        logger.info(f"   🚨 Emergency Mode: {'✅ Enabled' if cls.EMERGENCY_MODE else '❌ Disabled'}")

## 3. Manual Configuration (for Local or Colab Use)

This cell allows you to manually set your OpenRouter API key. **This is required to use the audio transcription feature.**

In [ ]:
# IMPORTANT: To use the audio transcription feature, you must provide your OpenRouter API key.
# You can get a free key from https://openrouter.ai/

# ----------------------------------------------------------------------------------
# --- PASTE YOUR OPENROUTER API KEY HERE ---
Config.OPENROUTER_API_KEY = ""
# ----------------------------------------------------------------------------------

# You can also override other settings if you wish:
# Config.AI_METHOD = "openrouter" # Recommended for Colab
# Config.OPENROUTER_MODEL = "google/gemma-2-9b-it:free"

# Display the final configuration
if not Config.OPENROUTER_API_KEY:
    logger.warning("🔴 OPENROUTER_API_KEY is not set! Audio transcription and OpenRouter models will not be available.")
else:
    logger.info("🟢 OPENROUTER_API_KEY has been set.")
    
Config.display()

## 4. System Prompts for Medical Consultation

The `MedicalSystemPrompts` class defines the instructional prompts for the AI, guiding its behavior in different medical contexts.

In [ ]:
class MedicalSystemPrompts:
    """Collection of system prompts for different medical scenarios"""
    
    BASE = """You are Dr. MediMate, an experienced physician providing direct medical consultations. You are the doctor the patient is seeing - not a referral service.

Your role as the attending physician:

1. **Conduct Medical Interviews**: Ask detailed follow-up questions to gather complete clinical information
2. **Perform Clinical Assessment**: Analyze symptoms systematically and provide working diagnoses
3. **Provide Direct Medical Care**: Give specific treatment recommendations, medications, and dosages
4. **Order Diagnostic Tests**: Recommend appropriate lab work, imaging, or procedures
5. **Manage Patient Care**: Provide ongoing medical management and follow-up plans

Your consultation approach:
- Act as the primary physician - YOU are the doctor they're consulting
- Ask detailed medical history questions (onset, duration, severity, triggers, alleviating factors)
- Inquire about associated symptoms, medical history, current medications, allergies
- Provide differential diagnoses with reasoning
- Give specific treatment plans including medications with exact dosages and schedules
- Recommend diagnostic tests when clinically indicated
- Provide clear medical advice and next steps
- Only suggest emergency care for true medical emergencies

Communication style:
- Professional but direct medical communication
- Ask targeted clinical questions to gather information
- Provide confident medical assessments
- Give specific, actionable medical recommendations
- Explain your clinical reasoning clearly

Remember: You ARE the doctor providing the consultation. Gather information, make assessments, and provide treatment recommendations like any physician would during a consultation."""

    EMERGENCY = BASE + """

IMPORTANT: You are handling a potential medical emergency. Prioritize assessment of life-threatening conditions and provide immediate guidance while recommending emergency services when appropriate. Be direct and clear in your instructions for both immediate first aid and when to call emergency services.

Emergency priorities:
- Assess ABC (Airway, Breathing, Circulation) status first
- Provide immediate first aid instructions when applicable
- Clearly state when emergency medical services (911/999/112) should be called
- Give specific instructions for what to do while waiting for emergency services
- Use direct, concise language appropriate for crisis situations"""

    PEDIATRIC = BASE + """

SPECIAL CONSIDERATIONS: You are treating a pediatric patient. Adjust your approach accordingly:
- Use age-appropriate assessment
- Calculate medication dosages based on weight when applicable
- Consider developmental factors in your evaluation
- Include guidance specifically for parents/caregivers
- Be especially vigilant about safety and emergency situations"""

    CHRONIC = BASE + """

FOCUS: You are managing a patient with chronic condition(s). Consider:
- Long-term medication management and adherence
- Disease progression monitoring
- Quality of life improvements
- Coordination with specialists
- Patient education for self-management
- Strategies to prevent complications"""

    @classmethod
    def get_prompt(cls, patient_type="standard", is_emergency=False):
        """Get the appropriate system prompt based on context"""
        if is_emergency:
            return cls.EMERGENCY
            
        if patient_type == "pediatric":
            return cls.PEDIATRIC
        elif patient_type == "chronic":
            return cls.CHRONIC
        else:
            return cls.BASE

## 5. AI Service Management

The `AIService` class handles connections to AI providers (Ollama or OpenRouter) and manages chat completions, including failover logic.

In [ ]:
class AIService:
    """Manages connections to AI providers and handles chat completions"""
    
    def __init__(self):
        self.openrouter_client = None
        self.ollama_available = False
        self.openrouter_available = False
        self.selected_method = None
        self.initialize_services()
        
    def initialize_services(self):
        """Initialize and test AI service connections"""
        logger.info("🔍 Checking AI service connections...")
        
        # Initialize OpenRouter if API key is available
        if Config.OPENROUTER_API_KEY:
            try:
                self.openrouter_client = OpenAI(
                    base_url=Config.OPENROUTER_BASE_URL,
                    api_key=Config.OPENROUTER_API_KEY,
                )
                logger.info("✅ OpenRouter client initialized")
            except Exception as e:
                logger.error(f"❌ Failed to initialize OpenRouter client: {e}")
                self.openrouter_client = None
        
        # Check service availability
        self.ollama_available = self._check_ollama_connection()
        self.openrouter_available = self._check_openrouter_connection()
        
        # Select method based on configuration and availability
        self._select_method()
        
    def _check_ollama_connection(self):
        """Check if Ollama service is running and accessible"""
        if not ollama:
            logger.warning("⚠️ Ollama module not available")
            return False
            
        try:
            response = requests.get(f"{Config.OLLAMA_HOST}/api/version", timeout=5)
            if response.status_code == 200:
                logger.info("✅ Ollama is running and accessible!")
                return True
            else:
                logger.warning(f"❌ Ollama responded with status code: {response.status_code}")
                return False
        except requests.exceptions.ConnectionError:
            logger.warning("❌ Cannot connect to Ollama!")
            logger.info("💡 Make sure to:")
            logger.info("   1. Install Ollama from https://ollama.ai/")
            logger.info("   2. Run 'ollama serve' in terminal")
            logger.info(f"   3. Download a model with 'ollama pull {Config.DEFAULT_MODEL}'")
            return False
        except Exception as e:
            logger.warning(f"❌ Error checking Ollama connection: {e}")
            return False
            
    def _check_openrouter_connection(self):
        """Check if OpenRouter API is accessible"""
        if not self.openrouter_client:
            logger.warning("❌ OpenRouter client not available (missing API key)")
            return False
            
        # Try multiple free models in case one is unavailable
        test_models = [
            "meta-llama/llama-3.2-9b-instruct:free",
            "meta-llama/llama-3.2-3b-instruct:free",
            "microsoft/phi-3-medium-128k-instruct:free", 
            "google/gemma-2-9b-it:free"
        ]
        
        for model in test_models:
            try:
                response = self.openrouter_client.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": "Test"}],
                    max_tokens=1
                )
                logger.info(f"✅ OpenRouter ready with model: {model}")
                # Update the global model if this one works and user used default
                if Config.OPENROUTER_MODEL == os.getenv("OPENROUTER_MODEL", "meta-llama/llama-3.2-9b-instruct:free"):
                    Config.OPENROUTER_MODEL = model
                return True
            except Exception as e:
                logger.warning(f"⚠️ Model {model} failed: {str(e)[:100]}...")
                continue
        
        logger.error("❌ OpenRouter: All test models failed")
        return False
        
    def _select_method(self):
        """Determine which AI method to use based on config and availability"""
        if Config.AI_METHOD == "ollama" and self.ollama_available:
            self.selected_method = "ollama"
            logger.info("🎯 Using: Ollama (Local)")
        elif Config.AI_METHOD == "openrouter" and self.openrouter_available:
            self.selected_method = "openrouter"
            logger.info("🎯 Using: OpenRouter (API)")
        elif Config.AI_METHOD == "auto":
            if self.openrouter_available: # Prioritize OpenRouter in auto mode if available
                self.selected_method = "openrouter"
                logger.info("🎯 Auto-selected: OpenRouter (API)")
            elif self.ollama_available:
                self.selected_method = "ollama"
                logger.info("🎯 Auto-selected: Ollama (Local)")
            else:
                self.selected_method = None
                logger.error("❌ No AI services available!")
        else:
            self.selected_method = None
            logger.error("❌ Selected AI method is not available!")
            
        # Additional logging for selected method
        if self.selected_method == "ollama" and self.ollama_available:
            try:
                models = ollama.list()
                logger.info(f"\n📋 Available Ollama models:")
                for model in models.get('models', []):
                    logger.info(f"   • {model.get('name', 'Unknown')}")
            except Exception as e:
                logger.warning(f"⚠️  Could not list Ollama models: {e}")
        elif self.selected_method == "openrouter":
            logger.info(f"\n🌐 Using OpenRouter model: {Config.OPENROUTER_MODEL}")
            
    def get_chat_completion(self, 
                           messages: List[Dict[str, str]], 
                           stream: bool = True, 
                           temperature: float = 0.7,
                           max_tokens: int = 1000) -> Generator[str, None, None]:
        """
        Get chat completion from the selected AI service
        
        Args:
            messages: List of message dictionaries with role and content
            stream: Whether to stream the response
            temperature: Temperature parameter for generation
            max_tokens: Maximum tokens to generate
            
        Yields:
            Partial response strings when streaming is enabled
        """
        if self.selected_method == "ollama" and self.ollama_available:
            yield from self._get_ollama_completion(messages, stream, temperature, max_tokens)
        elif self.selected_method == "openrouter" and self.openrouter_available:
            yield from self._get_openrouter_completion(messages, stream, temperature, max_tokens)
        else:
            yield "❌ No AI service is available. Please check your configuration and connections."
            
    def _get_ollama_completion(self, 
                              messages: List[Dict[str, str]], 
                              stream: bool = True,
                              temperature: float = 0.7,
                              max_tokens: int = 1000) -> Generator[str, None, None]:
        """Get completion from Ollama"""
        try:
            response_text = ""
            for chunk in ollama.chat(
                model=Config.DEFAULT_MODEL,
                messages=messages,
                stream=stream,
                options={
                    "temperature": temperature,
                    "num_predict": max_tokens
                }
            ):
                if 'message' in chunk and 'content' in chunk['message']:
                    token = chunk['message']['content']
                    response_text += token
                    yield response_text
                    
        except Exception as e:
            error_msg = f"❌ Ollama error: {str(e)}"
            logger.error(error_msg)
            yield error_msg
            
            # Try to fallback to OpenRouter if available
            if self.openrouter_available:
                fallback_msg = "\n\n💡 Trying OpenRouter as backup..."
                logger.info(fallback_msg)
                yield error_msg + fallback_msg
                yield from self._get_openrouter_completion(messages, stream, temperature, max_tokens)
            
    def _get_openrouter_completion(self, 
                                  messages: List[Dict[str, str]], 
                                  stream: bool = True,
                                  temperature: float = 0.7,
                                  max_tokens: int = 1000) -> Generator[str, None, None]:
        """Get completion from OpenRouter"""
        try:
            # Get response from OpenRouter
            response = self.openrouter_client.chat.completions.create(
                model=Config.OPENROUTER_MODEL,
                messages=messages,
                stream=stream,
                max_tokens=max_tokens,
                temperature=temperature,
                extra_headers={
                    "HTTP-Referer": "http://localhost:7860",
                    "X-Title": "MediMate Medical Assistant",
                }
            )
            
            response_text = ""
            for chunk in response:
                if chunk.choices and chunk.choices[0].delta.content:
                    token = chunk.choices[0].delta.content
                    response_text += token
                    yield response_text
                    
        except Exception as e:
            error_msg = f"❌ OpenRouter error: {str(e)}"
            logger.error(error_msg)
            yield error_msg
            
            # Try to fallback to Ollama if available
            if self.ollama_available:
                fallback_msg = "\n\n💡 Trying Ollama as backup..."
                logger.info(fallback_msg)
                yield error_msg + fallback_msg
                yield from self._get_ollama_completion(messages, stream, temperature, max_tokens)

## 6. Medical Consultation Logic

The `MedicalConsultation` class handles the core logic. It now prepends emergency warnings to the AI response stream for a smoother UI experience.

In [ ]:
class MedicalConsultation:
    """Handles medical consultation logic and session management"""
    
    def __init__(self, ai_service: AIService):
        self.ai_service = ai_service
        self.current_language = Config.DEFAULT_LANGUAGE
        self.patient_type = "standard"
        self.is_emergency = False
        
    def get_system_prompt(self):
        """Get the appropriate system prompt for the current consultation"""
        return MedicalSystemPrompts.get_prompt(
            patient_type=self.patient_type,
            is_emergency=self.is_emergency
        )
        
    def detect_language(self, text: str) -> str:
        """Detect the language of the input text"""
        if not Config.ENABLE_MULTILINGUAL:
            return Config.DEFAULT_LANGUAGE
            
        try:
            return detect(text)
        except:
            return Config.DEFAULT_LANGUAGE
            
    def detect_emergency(self, text: str) -> bool:
        """Detect if the message indicates a medical emergency"""
        if not Config.EMERGENCY_MODE:
            return False
            
        # Expanded list of keywords for better detection
        emergency_keywords = [
            "emergency", "severe pain", "chest pain", "heart attack", "stroke", 
            "can't breathe", "cannot breathe", "difficulty breathing", "unconscious", "collapsed",
            "bleeding heavily", "suicide", "overdose", "seizure", "not breathing",
            "choking", "major trauma", "head injury", "loss of consciousness", "am I having a stroke"
        ]
        
        text_lower = text.lower()
        for keyword in emergency_keywords:
            if keyword in text_lower:
                logger.warning(f"🚨 Emergency keyword detected: {keyword}")
                return True
                
        return False
        
    def detect_patient_type(self, text: str) -> str:
        """Detect the type of patient from the message"""
        text_lower = text.lower()
        
        # Check for pediatric patient
        pediatric_indicators = [
            "my child", "my baby", "my son", "my daughter", "infant", "toddler",
            "year old", "month old", "week old", "pediatric", "children's"
        ]
        for indicator in pediatric_indicators:
            if indicator in text_lower:
                return "pediatric"
                
        # Check for chronic condition patient
        chronic_indicators = [
            "chronic", "long-term", "ongoing", "years", "diabetes", "hypertension",
            "high blood pressure", "asthma", "copd", "arthritis", "long covid"
        ]
        for indicator in chronic_indicators:
            if indicator in text_lower:
                return "chronic"
                
        return "standard"
        
    def process_user_message(self, message: str, history: List[Tuple[str, str]]) -> Generator[str, None, None]:
        """
        Process a user message and generate a response
        
        Args:
            message: The user's message
            history: The conversation history
            
        Yields:
            The assistant's response, chunk by chunk
        """
        # Detect language, emergency status, and patient type for the current message
        self.current_language = self.detect_language(message)
        self.is_emergency = self.detect_emergency(message)
        # Don't override patient type if it was manually set
        if self.patient_type == "standard":
            self.patient_type = self.detect_patient_type(message)
        
        # Build conversation context
        conversation = [{"role": "system", "content": self.get_system_prompt()}]
        
        # Add chat history (limited to prevent context overflow)
        for user_msg, assistant_msg in history[-Config.MAX_HISTORY:]:
            conversation.append({"role": "user", "content": user_msg})
            # Remove any previous emergency prefixes from history to not confuse the model
            if assistant_msg.startswith("⚠️ **POTENTIAL EMERGENCY DETECTED**"):
                assistant_msg = assistant_msg.split("\n\n", 1)[-1]
            conversation.append({"role": "assistant", "content": assistant_msg})
        
        # Add current message
        conversation.append({"role": "user", "content": message})
        
        # Get response stream from AI service
        response_generator = self.ai_service.get_chat_completion(conversation)
        
        # Prepend emergency prefix to the stream if needed
        if self.is_emergency:
            emergency_prefix = "⚠️ **POTENTIAL EMERGENCY DETECTED** - Providing urgent assessment. **If this is a real emergency, call your local emergency services immediately.**\n\n"
            
            # Yield the prefix prepended to each chunk from the AI
            try:
                first_chunk = next(response_generator)
                yield emergency_prefix + first_chunk
                for chunk in response_generator:
                    yield emergency_prefix + chunk
            except StopIteration:
                # If the generator is empty, just yield the prefix
                yield emergency_prefix
        else:
            # If not an emergency, just yield the original generator
            yield from response_generator

## 7. PDF Export Functionality

The `PDFExporter` is updated to accept more metadata and provide a more informative report.

In [ ]:
class PDFExporter:
    """Handles exporting conversation to PDF"""
    
    @staticmethod
    def export_to_pdf(history: List[Tuple[str, str]], patient_type: str, is_emergency: bool, filename: str = None) -> str:
        """Export conversation history to a PDF file. Returns the file path."""
        if not Config.ENABLE_PDF_EXPORT:
            return "PDF export is disabled in configuration."
            
        try:
            # Create directory if it doesn't exist
            if not os.path.exists(Config.CONVERSATION_DIR):
                os.makedirs(Config.CONVERSATION_DIR)

            # Create PDF object
            pdf = FPDF()
            pdf.add_page()
            
            # Set up fonts
            pdf.set_font("Arial", "B", 16)
            pdf.cell(0, 10, "MediMate Medical Consultation Record", ln=True, align="C")
            pdf.set_font("Arial", "", 12)
            
            # Add metadata
            pdf.cell(0, 10, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}", ln=True)
            pdf.cell(0, 10, f"Patient Profile: {patient_type.capitalize()}", ln=True)
            if is_emergency:
                pdf.set_text_color(255, 0, 0)
                pdf.cell(0, 10, "Status: Potential Emergency Detected in Last Message", ln=True)
                pdf.set_text_color(0, 0, 0)
            pdf.ln(5)
            
            # Add conversation
            pdf.set_font("Arial", "B", 12)
            pdf.cell(0, 10, "Consultation Transcript:", ln=True)
            pdf.set_font("Arial", "", 10)
            
            for i, (user_msg, assistant_msg) in enumerate(history):
                pdf.set_font("Arial", "B", 10)
                pdf.multi_cell(0, 5, f"Patient:")
                pdf.set_font("Arial", "", 10)
                pdf.multi_cell(0, 5, user_msg)
                pdf.ln(3)
                
                pdf.set_font("Arial", "B", 10)
                pdf.multi_cell(0, 5, f"Dr. MediMate:")
                pdf.set_font("Arial", "", 10)
                pdf.multi_cell(0, 5, assistant_msg)
                pdf.ln(8)
            
            # Add disclaimer
            pdf.ln(5)
            pdf.set_font("Arial", "I", 8)
            pdf.multi_cell(0, 5, "Disclaimer: This consultation record is provided for informational purposes only and does not constitute medical advice. Please consult with a qualified healthcare professional for diagnosis and treatment of medical conditions.")
            
            # Generate filename if not provided
            if not filename:
                filename = os.path.join(
                    Config.CONVERSATION_DIR,
                    f"MediMate_Consultation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
                )
                
            # Save PDF
            pdf.output(filename)
            logger.info(f"✅ Consultation exported to {filename}")
            return filename
            
        except Exception as e:
            logger.error(f"❌ Error exporting to PDF: {e}")
            return f"❌ Failed to export conversation: {str(e)}"

## 8. User Interface with Gradio

The `MediMateUI` class is updated with a `gr.Audio` component and the logic to handle transcription.

In [ ]:
class MediMateUI:
    """Handles the Gradio UI for MediMate"""
    
    def __init__(self, consultation: MedicalConsultation):
        self.consultation = consultation
        self.interface = None
        
    def create_interface(self):
        """Create and configure the Gradio interface"""
        custom_css = """
        .gradio-container { max-width: 850px !important; margin: auto !important; }
        .disclaimer { background-color: #fff3cd; border: 1px solid #ffeaa7; border-radius: 5px; padding: 15px; margin: 10px 0; color: #856404; }
        .emergency-notice { background-color: #f8d7da; border: 1px solid #f5c6cb; border-radius: 5px; padding: 15px; margin: 10px 0; color: #721c24; font-weight: bold; }
        """
        
        ai_status = self.consultation.ai_service.selected_method
        theme = gr.themes.Soft(primary_hue="blue", secondary_hue="indigo")
        
        feature_desc = self._build_feature_description()
        examples = [
            "I have sharp chest pain for 2 hours, worse when breathing deeply. What do you think it is?",
            "Severe headaches every morning for 3 weeks, pounding pain. What tests should I get?",
            "My 5-year-old has 101°F fever, cough, runny nose for 2 days. What medicine should I give?",
            "I've been diagnosed with type 2 diabetes. What diet should I follow?",
        ]
        
        with gr.Blocks(theme=theme, css=custom_css, title="MediMate") as interface:
            gr.Markdown("<h1>🩺 MediMate - Professional AI Medical Consultation</h1>")
            
            with gr.Row():
                with gr.Column(scale=2):
                    gr.Markdown(feature_desc)
                with gr.Column(scale=1):
                    gr.Markdown("### 🔍 Current Configuration")
                    with gr.Group():
                        gr.Markdown(f"""
                        - 🎯 **AI Engine**: {ai_status.upper() if ai_status else "❌ UNAVAILABLE"}
                        - 🎤 **Whisper**: {'✅ Enabled' if self.consultation.ai_service.openrouter_client else '❌ Disabled'}
                        - 📄 **PDF Export**: {"✅ Enabled" if Config.ENABLE_PDF_EXPORT else "❌ Disabled"}
                        - 🚨 **Emergency Detection**: {"✅ Enabled" if Config.EMERGENCY_MODE else "❌ Disabled"}
                        """)
            
            gr.Markdown("<div class='emergency-notice'>⚠️ <strong>IMPORTANT MEDICAL DISCLAIMER</strong>: This AI is for informational purposes only. In case of a real medical emergency, call your local emergency services immediately (e.g., 911, 999, 112).</div>")
            
            chatbot = gr.Chatbot(label="Medical Consultation", height=500, show_copy_button=True, bubble_full_width=False)
            
            # Input area with Text and Audio options
            with gr.Group():
                with gr.Row():
                    msg = gr.Textbox(placeholder="Type your symptoms here...", label="Your Message (Text Input)", lines=3, scale=7)
                    audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Or Record Your Message (Audio Input)", scale=3)
                with gr.Row():
                    submit_btn = gr.Button("Send Message", variant="primary")

            with gr.Row():
                clear_btn = gr.Button("Start New Consultation", variant="secondary")
                
            with gr.Accordion("Advanced Options & Session Management", open=False):
                patient_type = gr.Radio(["standard", "pediatric", "chronic"], label="Patient Type", value="standard", info="Select patient type for specialized advice.")
                with gr.Row():
                    if Config.ENABLE_PDF_EXPORT:
                        export_pdf_btn = gr.Button("Export to PDF 📄", variant="secondary")
                    save_chat_btn = gr.Button("Save Chat 💾", variant="secondary")
                    load_chat_btn = gr.UploadButton("Load Chat 📂", file_types=[".json"])
                
                download_file = gr.File(label="Download Your Exported File", interactive=False)

            gr.Examples(examples=examples, inputs=msg, label="Example Medical Questions")
            
            # Event handlers
            submit_btn.click(self._chat_wrapper, [msg, chatbot], [msg, chatbot])
            audio_input.stop_recording(self._transcribe_and_chat, [audio_input, chatbot], [msg, chatbot, audio_input])

            clear_btn.click(lambda: (None, None, "standard", None), None, [msg, chatbot, patient_type, audio_input])
            patient_type.change(self._update_patient_type, patient_type, None)
            
            if Config.ENABLE_PDF_EXPORT:
                export_pdf_btn.click(self._export_consultation, [chatbot], [download_file])
            
            save_chat_btn.click(self._save_consultation, [chatbot], [download_file])
            load_chat_btn.upload(self._load_consultation, [load_chat_btn], [chatbot])
                
        self.interface = interface
        return interface
    
    def _build_feature_description(self):
        ai_status = self.consultation.ai_service.selected_method
        status_color = "#e3f2fd" if ai_status else "#f8d7da"
        status_text = "🖥️ Running locally via Ollama" if ai_status == 'ollama' else "🌐 Using OpenRouter API" if ai_status == 'openrouter' else "❌ No AI service available"
        return f"""
        <div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; border-left: 4px solid #007bff;'>
        <h3>👨‍⚕️ Welcome to MediMate</h3>
        <p>Describe your symptoms by typing or recording audio for a professional medical assessment.</p></div>
        <div style='background-color: {status_color}; padding: 10px; border-radius: 5px; margin-top: 10px;'>
        <p><b>AI Service Status:</b> {status_text}</p></div>
        """
    
    def _transcribe_and_chat(self, audio_path, history):
        """Transcribes audio and then passes the text to the chat wrapper."""
        if not audio_path:
            return "", history, None
            
        client = self.consultation.ai_service.openrouter_client
        if not client:
            gr.Warning("Audio transcription requires an OpenRouter API key. Please configure it and restart.")
            return "", history, None

        gr.Info("Transcribing audio... Please wait.")
        try:
            with open(audio_path, "rb") as audio_file:
                transcription = client.audio.transcriptions.create(
                    model=Config.WHISPER_MODEL,
                    file=audio_file
                )
            transcribed_text = transcription.text
            gr.Info("Transcription successful! Sending to AI...")

            # Now, call the chat wrapper with the transcribed text
            # We need to handle the generator returned by _chat_wrapper
            final_msg, final_history = "", history
            for msg_chunk, history_chunk in self._chat_wrapper(transcribed_text, history):
                final_msg, final_history = msg_chunk, history_chunk
                yield final_msg, final_history, None # Update UI progressively

            yield "", final_history, None # Final update to clear textbox

        except Exception as e:
            logger.error(f"❌ Error during audio transcription: {e}")
            gr.Error(f"Audio transcription failed: {e}")
            yield "", history, None

    def _chat_wrapper(self, message, history):
        if not message or message.strip() == "":
            gr.Warning("Please enter a message.")
            yield "", history
            return
            
        history = history or []
        history.append((message, ""))
        
        response_generator = self.consultation.process_user_message(message, history[:-1])
        
        for chunk in response_generator:
            history[-1] = (message, chunk)
            yield "", history
            
    def _update_patient_type(self, patient_type):
        self.consultation.patient_type = patient_type
        gr.Info(f"Patient type set to: {patient_type.capitalize()}")
        
    def _export_consultation(self, history):
        if not history:
            gr.Warning("No consultation to export.")
            return None
        
        filepath = PDFExporter.export_to_pdf(
            history, 
            self.consultation.patient_type, 
            self.consultation.is_emergency
        )
        
        if "❌" in filepath:
            gr.Error(filepath)
            return None
        
        gr.Info("PDF export successful!")
        return filepath

    def _save_consultation(self, history):
        if not history:
            gr.Warning("No consultation to save.")
            return None
        
        if not os.path.exists(Config.CONVERSATION_DIR):
            os.makedirs(Config.CONVERSATION_DIR)
            
        filename = os.path.join(Config.CONVERSATION_DIR, f"MediMate_Chat_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        with open(filename, 'w') as f:
            json.dump(history, f, indent=2)
            
        gr.Info(f"Conversation saved!")
        return filename

    def _load_consultation(self, file):
        if not file:
            gr.Warning("File upload failed.")
            return None
        try:
            with open(file.name, 'r') as f:
                history = json.load(f)
            gr.Info(f"Conversation loaded from {os.path.basename(file.name)}")
            return history
        except Exception as e:
            gr.Error(f"Failed to load or parse JSON file: {e}")
            return None
        
    def launch(self, **kwargs):
        if not self.interface:
            self.create_interface()
        
        # Automatically enable sharing if in Google Colab
        is_colab = 'google.colab' in sys.modules
        share_ui = kwargs.get('share', is_colab)
        if is_colab:
            logger.info("✅ Running in Google Colab. Public link will be generated.")
            
        try:
            self.interface.launch(
                server_name=kwargs.get("server_name", "0.0.0.0"),
                server_port=kwargs.get("server_port", Config.SERVER_PORT),
                share=share_ui,
                quiet=kwargs.get("quiet", False),
                show_error=kwargs.get("show_error", True),
                inbrowser=kwargs.get("inbrowser", True)
            )
        except Exception as e:
            logger.error(f"❌ Error launching interface: {e}")

## 9. Main Execution Function

This function initializes and runs the MediMate system.

In [ ]:
def run_medimate():
    """Main function to run MediMate"""
    logger.info("🚀 Starting MediMate - Enhanced AI Medical Consultation System")
    logger.info("="*60)
    
    # Initialize AI service
    ai_service = AIService()
    
    if not ai_service.selected_method:
        logger.error("❌ MediMate cannot start - No AI backend available")
        logger.info("\n🔧 Setup Options:")
        logger.info("   1. For Colab/API use: Set your OPENROUTER_API_KEY in cell #3.")
        logger.info("   2. For local use: Install and run Ollama.")
        return
    
    # Initialize consultation manager
    consultation = MedicalConsultation(ai_service)
    
    # Create and launch UI
    ui = MediMateUI(consultation)
    ui.create_interface()
    
    logger.info("="*60)
    logger.info("👨‍⚕️ Dr. MediMate is ready for consultations...")
    logger.info("="*60)
    
    # Launch the interface
    ui.launch()
    
    return ui.interface

## 10. Launch MediMate

Execute the cell below to start the application. If you are in Google Colab, a public link will be generated to access the UI.

In [ ]:
# Run MediMate
# This condition ensures it runs automatically in Colab but not if imported as a module.
if __name__ == "__main__" and 'google.colab' in sys.modules:
    run_medimate()